# Car Price Prediction Model 

In [104]:
#IMPORTING LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [105]:
#IMPORTING DATASET
dataset = pd.read_csv('/home/bravin/Documents/Practice/Car-Price-Prediction-Challenge/car_price_prediction.csv',index_col=0)

#CHECKING DATASET
dataset.head()

,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
ID,,,,,,,,,,,,,,,,,
45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


In [106]:
#CHECKING DATASET SHAPE
dataset.shape

(19237, 17)

In [107]:
#CHECKING DATASET INFO
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19237 entries, 45654403 to 45813273
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Price             19237 non-null  int64  
 1   Levy              19237 non-null  object 
 2   Manufacturer      19237 non-null  object 
 3   Model             19237 non-null  object 
 4   Prod. year        19237 non-null  int64  
 5   Category          19237 non-null  object 
 6   Leather interior  19237 non-null  object 
 7   Fuel type         19237 non-null  object 
 8   Engine volume     19237 non-null  object 
 9   Mileage           19237 non-null  object 
 10  Cylinders         19237 non-null  float64
 11  Gear box type     19237 non-null  object 
 12  Drive wheels      19237 non-null  object 
 13  Doors             19237 non-null  object 
 14  Wheel             19237 non-null  object 
 15  Color             19237 non-null  object 
 16  Airbags           19237 non-nu

In [108]:
# REMOVING KM FROM MILEAGE
dataset.Mileage = dataset.Mileage.str.replace('km','')

In [109]:
# CHANGING Prod Year to Date
dataset['Prod. year'] = pd.to_datetime(dataset['Prod. year'], format='%Y')

In [110]:
# Investgate Engine Volume 
dataset['Engine volume'].value_counts()

# REmove Turbo from Engine Volume
dataset['Engine volume'] = dataset['Engine volume'].str.replace('Turbo','')

# Convert Engine Volume to Float
dataset['Engine volume'] = dataset['Engine volume'].astype(float)

In [111]:
# Convert Mileage to Float
dataset['Mileage'] = dataset['Mileage'].astype(float)

In [112]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19237 entries, 45654403 to 45813273
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Price             19237 non-null  int64         
 1   Levy              19237 non-null  object        
 2   Manufacturer      19237 non-null  object        
 3   Model             19237 non-null  object        
 4   Prod. year        19237 non-null  datetime64[ns]
 5   Category          19237 non-null  object        
 6   Leather interior  19237 non-null  object        
 7   Fuel type         19237 non-null  object        
 8   Engine volume     19237 non-null  float64       
 9   Mileage           19237 non-null  float64       
 10  Cylinders         19237 non-null  float64       
 11  Gear box type     19237 non-null  object        
 12  Drive wheels      19237 non-null  object        
 13  Doors             19237 non-null  object        
 14  Wheel       

In [116]:
#Cheking null values
dataset.isnull().sum()

Price               0
Levy                0
Manufacturer        0
Model               0
Prod. year          0
Category            0
Leather interior    0
Fuel type           0
Engine volume       0
Mileage             0
Cylinders           0
Gear box type       0
Drive wheels        0
Doors               0
Wheel               0
Color               0
Airbags             0
dtype: int64

In [113]:
# DESCRIBE DATASET
dataset.describe()

,Price,Engine volume,Mileage,Cylinders,Airbags
count,1.923700e+04,19237.000000,1.923700e+04,19237.000000,19237.000000
mean,1.855593e+04,2.307990,1.532236e+06,4.582991,6.582627
std,1.905813e+05,0.877805,4.840387e+07,1.199933,4.320168
min,1.000000e+00,0.000000,0.000000e+00,1.000000,0.000000
25%,5.331000e+03,1.800000,7.013900e+04,4.000000,4.000000
50%,1.317200e+04,2.000000,1.260000e+05,4.000000,6.000000
75%,2.207500e+04,2.500000,1.888880e+05,4.000000,12.000000
max,2.630750e+07,20.000000,2.147484e+09,16.000000,16.000000


In [114]:
# CHECKING NULL VALUES
dataset.isnull().sum()

Price               0
Levy                0
Manufacturer        0
Model               0
Prod. year          0
Category            0
Leather interior    0
Fuel type           0
Engine volume       0
Mileage             0
Cylinders           0
Gear box type       0
Drive wheels        0
Doors               0
Wheel               0
Color               0
Airbags             0
dtype: int64